<a href="https://colab.research.google.com/github/maivan-hoa/Sentiment-Classification/blob/main/Setiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
# tải tập dữ liệu imdb review
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZD2I69/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZD2I69/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZD2I69/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']


In [ ]:
print(train_data)

<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>


In [ ]:
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

# dữ liệu lưu trong train_data cà test_data phải lấy ra từng cái một
for s, l in train_data:
  # s là mảng tensor, cần convert to numpy rồi sang string
  train_sentences.append(str(s.numpy()))
  train_labels.append(l.numpy())

for s, l in test_data:
  test_sentences.append(str(s.numpy()))
  test_labels.append(l.numpy())

In [ ]:
for s, l in train_data:
  print(str(s.numpy()))
  print(l)
  break

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
tf.Tensor(0, shape=(), dtype=int64)


In [ ]:
type(train_sentences)

list

In [ ]:
# chuyển nhãn thành mảng numpy
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
# chỉ định các siêu tham số
vocab_size = 10000
embedding_dim = 64
max_length = 140

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(train_sentences)

In [ ]:
tokenizer.word_index

In [ ]:
train_sentences = tokenizer.texts_to_sequences(train_sentences)

In [ ]:
train_sentences

In [ ]:
padded_train_sentences = pad_sequences(train_sentences, maxlen=max_length, truncating='post', padding='post') # truncating: nếu câu dài quá maxlen thì bị cắt ở đằng sau

In [ ]:
padded_train_sentences

array([[  59,   12,   14, ...,    0,    0,    0],
       [ 256,   28,   78, ...,    0,    0,    0],
       [   1, 6175,    2, ...,    0,    0,    0],
       ...,
       [7629,   37,   11, ..., 2743,   21, 1563],
       [2677,   11,  216, ..., 1048,   37,   31],
       [3875,    5,   31, ...,    1, 2238,   42]], dtype=int32)

In [ ]:
padded_train_sentences.shape

(25000, 140)

In [ ]:
test_sentences = tokenizer.texts_to_sequences(test_sentences)
padded_test_sentences = pad_sequences(test_sentences, maxlen=max_length, truncating='post', padding='post')

In [ ]:
type(padded_test_sentences)

numpy.ndarray

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(10, activation='relu'))

In [ ]:
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 140, 64)           640000    
_________________________________________________________________
flatten (Flatten)            (None, 8960)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                89610     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 729,621
Trainable params: 729,621
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(padded_train_sentences, train_labels, epochs=10, validation_data=(padded_test_sentences, test_labels)) # mặc định số batchsize là 32

Epoch 1/10
782/782 [==============================] - 10s 12ms/step - loss: 0.4480 - accuracy: 0.7870 - val_loss: 0.4276 - val_accuracy: 0.7987
Epoch 2/10
782/782 [==============================] - 10s 12ms/step - loss: 0.2272 - accuracy: 0.9144 - val_loss: 0.4271 - val_accuracy: 0.8070
Epoch 3/10
782/782 [==============================] - 10s 12ms/step - loss: 0.0781 - accuracy: 0.9796 - val_loss: 0.5010 - val_accuracy: 0.8062
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.0154 - accuracy: 0.9982 - val_loss: 0.5727 - val_accuracy: 0.8104
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 0.0034 - accuracy: 0.9999 - val_loss: 0.6257 - val_accuracy: 0.8129
Epoch 6/10
782/782 [==============================] - 10s 12ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.6694 - val_accuracy: 0.8144
Epoch 7/10
782/782 [==============================] - 10s 12ms/step - loss: 7.2441e-04 - accuracy: 1.0000 - val_loss: 0.7077 - val_accur

In [ ]:
train_labels.shape

(25000,)

In [ ]:
test_sen = ['I love this movie']
test_sen = tokenizer.texts_to_sequences(test_sen)
padded_test_sen = pad_sequences(test_sen, maxlen=max_length, truncating='post', padding='post')

In [ ]:
padded_test_sen

array([[ 11, 119,  12,  18,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [ ]:
model.predict(padded_test_sen)

array([[0.8181989]], dtype=float32)

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]

In [ ]:
weights

array([[-0.02271744, -0.00986559,  0.01660133, ..., -0.0012377 ,
        -0.01488584, -0.00012071],
       [ 0.03183817, -0.01344763,  0.02299856, ..., -0.06083233,
        -0.03556907,  0.01854963],
       [ 0.047151  , -0.00366081,  0.06131275, ..., -0.01480645,
        -0.04127599,  0.00671785],
       ...,
       [ 0.04649764, -0.01874455, -0.03202019, ..., -0.0362942 ,
         0.01897969,  0.05248039],
       [ 0.01354761, -0.00248087, -0.02399903, ..., -0.12065893,
         0.03514304, -0.03053501],
       [ 0.01338299, -0.10793795, -0.02627972, ..., -0.01426919,
        -0.01413884, -0.01095904]], dtype=float32)

In [ ]:
weights.shape

(10000, 64)

In [ ]:
import io

In [ ]:
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for idx in range(1, vocab_size):
  word = tokenizer.index_word[idx]
  vector = weights[idx]
  
  out_m.write(word + '\n')
  out_v.write('\t'.join([str(x) for x in vector]) + '\n')
  
out_m.close()
out_v.close()

# tải 2 file meta.tsv và vecs.tsv về rồi load lên ProjectorTensorflow